# Functions

In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import csv

def print_pixel_over_ROI(x,y,w,h,image_gray):
    """
    Print pixel values over the specified region of interest (ROI).

    Args:
        x (int): X-coordinate of the top-left corner of the ROI.
        y (int): Y-coordinate of the top-left corner of the ROI.
        w (int): Width of the ROI.
        h (int): Height of the ROI.
        image_gray (numpy.ndarray): Grayscale image.

    Returns:
        None
    """
    for row in range(y, y + h):
        for col in range(x, x + w):
            pixel = image_gray[row, col]
            print('Pixel at ({}, {}): {}'.format(row, col, pixel))
            
def extract_middle_roi(image, target_width):
    """
    Extract the middle region of an image with a specified target width.

    Args:
        image (numpy.ndarray): Input image.
        target_width (int): Desired width of the extracted region.

    Returns:
        numpy.ndarray: Extracted middle region of the image.
    """
    height, width = image.shape[:2]
    aspect_ratio = width / height
    target_height = int(target_width / aspect_ratio)
    x = int((width - target_width) / 2)
    y = int((height - target_height) / 2)
    x2 = x + target_width
    y2 = y + target_height
    middle_roi = image[y:y2, x:x2]
    return middle_roi

def filter_contours(image_path, output_path, ratio, ratio2=None):
    """
    Apply contour filtering based on aspect ratio and save the resulting image.

    Args:
        image_path (str): Path to the input grayscale image.
        output_path (str): Path to save the resulting filtered image.
        ratio (list): List with lower and upper bounds of aspect ratio.
        ratio2 (list, optional): Second set of aspect ratio bounds for filtering.

    Returns:
        numpy.ndarray: Filtered image with contours.
    """
    # Load the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the original image
    image_with_rectangles = np.copy(image)
    
    #ratio is a list with first element as lower bound and second element as upper bound
    aspect_ratios = []
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_with_rectangles, (x, y), (x + w, y + h), (255), 1)  # Adjust color and thickness as needed
        
        aspect_ratio = float(w) / h
        if not ratio2:
            if aspect_ratio >= ratio[0] and aspect_ratio <= ratio[1]:
                filtered_contours.append(contour)
        else:
            if (aspect_ratio >= ratio[0] and aspect_ratio <= ratio[1]) or (aspect_ratio >= ratio2[0] and aspect_ratio <= ratio2[1]):
                filtered_contours.append(contour)
                
    #Check the images with bounding rectangles
    cv2.imwrite("bounding_rect.jpg", image_with_rectangles)
    
    # Create a mask of the filtered contours
    mask = np.zeros_like(image)
    cv2.drawContours(mask, filtered_contours, -1, (255), thickness=cv2.FILLED)
    
    # Save the filtered image to the specified output path
    cv2.imwrite(output_path, mask)
    return mask

def preprocess_image(image_path, threshold_val=0):
    """
    Preprocess an image by cropping and thresholding.

    Args:
        image_path (str): Path to the input grayscale image.
        threshold_val (int, optional): Threshold value for image thresholding.

    Returns:
        tuple: Tuple containing threshold value and path to the preprocessed image.
    """
    # Load the grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Crop image
    image = extract_middle_roi(image, 1500)

    if threshold_val == 0:
        threshold_val = 5
        # Find threshold with sd
        for x in range(5, 10):
            mask = np.where(image < x, 1, 0)
            masked_intensities = image[mask == 1]
            mean_intensity = np.mean(masked_intensities)
            sd_intensity = np.std(masked_intensities)
            two_sd = (mean_intensity + (2 * sd_intensity))
            three_sd = (mean_intensity + (3 * sd_intensity))
            print("current threshold: {}".format(x))
            print("two_sd: {}".format(two_sd))
            print("three_sd: {}".format(three_sd))

            if x >= two_sd and x <= three_sd:
                threshold_val = x
                break

    # Filter out pixels with intensities below the threshold_val
    _, image_filtered = cv2.threshold(image, threshold_val, 255, cv2.THRESH_TOZERO)

    print("Threshold value:", threshold_val)

    # Save and download to local disk
    # Extract the filename from the original image path
    filename = image_path.split("/")[-1]

    # Create the new path by concatenating the desired directory and the filename
    new_path = os.path.dirname(image_path) + "/thresholded/"

    # Create the folder if it doesn't exist
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    new_path += filename

    cv2.imwrite(new_path, image_filtered)

    print("Saved thresholded image to: {}".format(new_path))

    return threshold_val, new_path

def get_max_intensity_location(image_name):
    # Load the grayscale image
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)

    # Initialize variables to track the highest intensity and its location
    max_intensity = 0
    max_intensity_location = None

    # Iterate over each pixel in the image
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            intensity = image[x,y]

            # Update the highest intensity and its location if a higher intensity is found
            if intensity > max_intensity and intensity < 255:
                max_intensity = intensity
                max_intensity_location = (x,y)
    
    # Print the highest intensity pixel
    print("Highest intensity:", max_intensity)
    
    # Print the location of the highest intensity pixel
    print("Highest intensity location:", max_intensity_location)
    return max_intensity_location

In [3]:
from PIL import Image
import math

def plot_intensity_vs_x(image_name, x, fixed_y):
    # Load the grayscale image
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)

    # Initialize lists to store the x-axis coordinates and y-axis intensities
    x_coordinates = []
    intensities = []

    # Iterate over each pixel in the image
    for x in range(x - 40, x + 40):
        intensity = image[fixed_y, x]
        x_coordinates.append(x)
        intensities.append(intensity)
        if intensity > 225:
            with open(os.path.dirname(image_name) + 'logfile.txt', "a") as f:
                print("{} is overexposed.".format(image_name), file=f)

    # Plot the graph
    plt.scatter(x_coordinates, intensities)
    plt.xlabel('X Coordinates')
    plt.ylabel('Pixel Intensity')
    plt.title('Intensity vs X Coordinate')

    plot_path = os.path.dirname(image_name) + "/plots/"

    # Create the folder if it doesn't exist
    if not os.path.exists(plot_path):
        os.makedirs(plot_path)

    plt.savefig(os.path.dirname(image_name) + "/plots/" + os.path.basename(image_name) + "_vs_x.jpg")

    plt.show()

def plot_intensity_vs_y(image_name, fixed_x, y):
    # Load the grayscale image
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)

    # Initialize lists to store the x-axis coordinates and y-axis intensities
    y_coordinates = []
    intensities = []

    # Iterate over each pixel in the image
    for y in range(y - 40, y + 40):
        intensity = image[y, fixed_x]
        y_coordinates.append(y)
        intensities.append(intensity)
        if intensity > 225:
            with open(os.path.dirname(image_name) + 'logfile.txt', "a") as f:
                print("{} is overexposed.".format(image_name), file=f)

    # Plot the graph
    plt.scatter(y_coordinates, intensities)
    plt.xlabel('Y Coordinates')
    plt.ylabel('Pixel Intensity')
    plt.title('Intensity vs Y Coordinate')

    plot_path = os.path.dirname(image_name) + "/plots/"

    # Create the folder if it doesn't exist
    if not os.path.exists(plot_path):
        os.makedirs(plot_path)

    plt.savefig(os.path.dirname(image_name) + "/plots/" + os.path.basename(image_name) + "_vs_y.jpg")

    plt.show()

# Calculate the average pixel intensity in each contour, and then the sum(average_intensities)/num contours
def get_contour_average_intensity(image_path, threshold):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Threshold the image
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_TOZERO)
    
    # Find contours in the image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # List to store average intensity values for each contour
    average_intensities = []
    
    # Loop through each contour
    for contour in contours:
        # Create a mask for the current contour
        mask = np.zeros_like(image)
        cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED)
        
        # Calculate the masked average intensity
        masked_intensities = image[mask == 255]
        if len(masked_intensities) > 0:
            average_intensity = np.mean(masked_intensities)
            average_intensities.append(average_intensity)
    
    return sum(average_intensities)/len(contours)

# Calculate the average number of white pixels in each contour
def get_contour_average_white_pixels(image_path, threshold):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Threshold the image to create a binary image
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    
    cv2.imwrite("binary_image.jpg", binary_image)
    
    # Find contours in the image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    total_white_pixels = 0
    total_white_pixels_arr = []
    total_contours = 0
    
    for contour in contours:
        white_pixels = cv2.contourArea(contour)
        if white_pixels>=10:
            total_white_pixels += white_pixels
            total_white_pixels_arr.append(white_pixels)
            total_contours += 1
            
#     # Sample data for the scatter plot
#     x_values = [x for x in range(len(total_white_pixels_arr))]
    
#     # Create the scatter plot
#     plt.scatter(x_values[:100], total_white_pixels_arr[:100])

#     # Add labels and title
#     plt.xlabel("X-axis label")
#     plt.ylabel("Y-axis label")
#     plt.title("Scatter Plot Example")

#     # Show the plot
#     plt.show()
    
    average_white_pixels = total_white_pixels / total_contours

    print("Average number of white pixels in each contour:", average_white_pixels)
    return average_white_pixels
    
    
def find_contour_closest_to_pixel(image_path, target_pixel_location):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Find contours in the image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate the centroid of the target pixel location
    target_centroid = np.array(target_pixel_location)
    
    closest_contour = None
    closest_contour_centroid = None
    min_distance = float('inf')
    
    for contour in contours:
        # Calculate the centroid of the current contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            centroid_x = int(M["m10"] / M["m00"])
            centroid_y = int(M["m01"] / M["m00"])
            centroid = np.array((centroid_x, centroid_y))

            # Calculate the Euclidean distance between the target pixel and the contour centroid
            distance = np.linalg.norm(target_centroid - centroid)

            # Update the closest contour if a smaller distance is found
            if distance < min_distance:
                closest_contour = contour
                min_distance = distance
                closest_contour_centroid = centroid

    x, y, w, h = cv2.boundingRect(closest_contour)

    # Draw the bounding rectangle on the image
    cv2.rectangle(image, (x, y), (x + w, y + h), (255), 2)
    
    # Draw the circle on the center of the contour
    cv2.circle(image, (closest_contour_centroid[0], closest_contour_centroid[1]), 3, (255), -1)
                
    print("Save test image")
    plot_path = os.path.dirname(image_path) + "/test_image/"

    # Create the folder if it doesn't exist
    if not os.path.exists(plot_path):
        os.makedirs(plot_path)

    # Save the image using OpenCV's imwrite()
    cv2.imwrite(os.path.join(plot_path, os.path.basename(image_path) + ".jpg"), image)
        
    return closest_contour, closest_contour_centroid

# Example usage
# target_pixel_location = (100, 200)  # Replace with the coordinates of your target pixel
# contour = find_contour_with_pixel("path/to/your/image.jpg", target_pixel_location)
# if contour is not None:
#     print("Contour found!")
# else:
#     print("No contour found containing the target pixel.")



In [4]:
import cv2
import numpy as np

def get_pixel_locations(image_path, threshold):
    # Read the original grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Create a binary mask where pixels above the threshold are white (255) and others are black (0)
    binary_mask = np.where(image > threshold, 255, 0).astype(np.uint8)

    # Find the pixel locations where the pixel intensities are above the threshold
    locations = np.column_stack(np.where(binary_mask > 0))

    return locations

def make_other_pixels_white(image_path, black_pixel_locations, output_path):
    # Read the original grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Create a binary mask where pixels in black_pixel_locations are black (0) and others are white (255)
    binary_mask = np.ones_like(image, dtype=np.uint8) * 255
    for loc in black_pixel_locations:
        row, col = loc
        binary_mask[row, col] = 0

    # Use the binary mask to set unspecified pixels to white (255) in the result image
    result_image = cv2.bitwise_and(image, binary_mask)

    # Save the resulting image with unspecified pixels turned to white
    cv2.imwrite(output_path, result_image)

def make_other_pixels_black(image_path, white_pixel_locations, output_path):
    # Read the original grayscale image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Create a binary mask where pixels in white_pixel_locations are white (255) and others are black (0)
    binary_mask = np.zeros_like(image, dtype=np.uint8)
    for loc in white_pixel_locations:
        row, col = loc
        binary_mask[row, col] = 255

    # Use the binary mask to set unspecified pixels to black (0) in the result image
    result_image = cv2.bitwise_and(image, binary_mask)

    # Save the resulting image with unspecified pixels turned to black
    cv2.imwrite(output_path, result_image)

def get_RGB_pixel_locations(red_image, green_image, blue_image, threshold):
    
    # Get the pixel locations with intensities above the threshold
    red_pixel_locations = get_pixel_locations(red_image, threshold)
    green_pixel_locations = get_pixel_locations(green_image, threshold)
    blue_pixel_locations = get_pixel_locations(blue_image, threshold)
    
    # Save the pixel locations as text files
    np.savetxt("red_pixel_locations.txt", red_pixel_locations, fmt="%d")
    np.savetxt("green_pixel_locations.txt", green_pixel_locations, fmt="%d")
    np.savetxt("blue_pixel_locations.txt", blue_pixel_locations, fmt="%d")
    
    print("Attained red, blue and green pixel locations")
    
    return red_pixel_locations, green_pixel_locations, blue_pixel_locations

In [5]:
#check whether two grayscale images are identical

def display_nonzero_numbers(array, max_count=100):
    # Get the indices of the nonzero elements
    indices = np.nonzero(array)

    # Extract the values at the nonzero indices
    values = array[indices]

    # Display up to the specified number of nonzero values
    count = min(max_count, len(values))
    for i in range(count):
        index = tuple(idx[i] for idx in indices)
        value = values[i]
        print(f"Value of diff array at index {index}: {value}")
        
def are_images_identical(image1, image2):
    # Compare the dimensions of the images
    if image1.shape != image2.shape:
        return False

    # Compare each pixel of the images
    difference = cv2.subtract(image1, image2)
    
    # Check if all pixels are zero (indicating identical images)
    display_nonzero_numbers(difference)
    return cv2.countNonZero(difference) == 0

# Sample use case

# image1 = cv2.imread("/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/mask_test_0731/Blue32/thresholded/blue/32_128_32_clear_5600000.bmp", cv2.IMREAD_GRAYSCALE)
# image2 = cv2.imread("/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/mask_test_0731/Blue32/thresholded/blue/128_32_32_clear_5600000.bmp", cv2.IMREAD_GRAYSCALE)

# are_images_identical(image1, image2)

# Driver Code

In [7]:
"""
Attain locations of RGB pixels

"""
# Use middle ROI, and threshold for 255s
red_image = "/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/leakage_dataRed8/8_8_8_X_70000000.bmp"
green_image = "/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/GL 255/255_255_255_Y_25000.bmp"
blue_image = "/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/GL 255/255_255_255_Z_40000.bmp"
threshold_value = 0

# Preprocess the red image
_, new_red_path = preprocess_image(red_image, threshold_val=threshold_value)

# Preprocess the green image
_, new_green_path = preprocess_image(green_image, threshold_val=threshold_value)

# Preprocess the blue image
_, new_blue_path = preprocess_image(blue_image, threshold_val=threshold_value)

# Iterate over the contours and filter out ovals with a width-to-length ratio:
red_ratio = [1.5, 5]
green_ratio = [1.3,10]
blue_ratio = [1, 1.5]

filter_contours(new_red_path, "new_red_image.jpg", red_ratio)
# filter_contours(new_blue_path, "new_blue_image.jpg", blue_ratio)
# filter_contours(new_green_path, "new_green_image.jpg", green_ratio)

# After filtering, attain locations of RGB pixels
red_image = "new_red_image.jpg"
green_image = "new_green_image.jpg"
blue_image = "new_blue_image.jpg"
threshold_value = 5

get_RGB_pixel_locations(red_image, green_image, blue_image, threshold_value)

current threshold: 5
two_sd: 3.34078756159091
three_sd: 4.188506874803132
current threshold: 6
two_sd: 3.7948386269697405
three_sd: 4.816116946558398
current threshold: 7
two_sd: 4.276928505906371
three_sd: 5.485605582326025
current threshold: 8
two_sd: 4.753283374770707
three_sd: 6.148924712323457
current threshold: 9
two_sd: 5.2423079339223975
three_sd: 6.831107520295996
Threshold value: 5
Saved thresholded image to: /mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/leakage_dataRed8/thresholded/8_8_8_X_70000000.bmp
current threshold: 5
two_sd: 3.681715480615872
three_sd: 4.530390357524872
current threshold: 6
two_sd: 4.224067149539939
three_sd: 5.272113011748198
current threshold: 7
two_sd: 4.912458125348822
three_sd: 6.21647128188323
current threshold: 8
two_sd: 5.777219595204662
three_sd: 7.400006713946173
current threshold: 9
two_sd: 6.854019967414025
three_sd: 8.864784936940406
Threshold value: 5
Saved thresholded image to: /mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/GL

(array([[   0,    0],
        [   0,    1],
        [   0,    2],
        ...,
        [1127, 1473],
        [1127, 1474],
        [1127, 1475]]),
 array([[   0,  105],
        [   0,  106],
        [   0,  107],
        ...,
        [1127, 1321],
        [1127, 1322],
        [1127, 1323]]),
 array([[   0,    0],
        [   0,    1],
        [   0,    2],
        ...,
        [1127, 1133],
        [1127, 1134],
        [1127, 1135]]))

In [ ]:
"""
Using RGB pixel locations, implement RGB filters

"""
# Read the pixel locations from the text files
red_pixel_locations = np.loadtxt("red_pixel_locations.txt", dtype=int)
green_pixel_locations = np.loadtxt("green_pixel_locations.txt", dtype=int)
blue_pixel_locations = np.loadtxt("blue_pixel_locations.txt", dtype=int)

# directory should include all images one wishes to process
directory = "/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/mask_test_0731/Blue32/"

image_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith((".jpg", ".jpeg", ".png", ".bmp"))]

# Preprocess images by thresholding them
threshold_val = 0
for image_file in image_files:
    if threshold_val == 0:
        threshold_val, _ = preprocess_image(image_file)
    else:
        preprocess_image(image_file, threshold_val=threshold_val)

# Read in images from corresponding directory
directory = directory + "thresholded/"

image_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith((".jpg", ".jpeg", ".png", ".bmp"))]

# Define the output folder names
output_folders = {
    "red": "red",
    "blue": "blue",
    "green": "green"
}

for original_image_path in image_files:
    # Get the filename from the original image path
    filename = os.path.basename(original_image_path)

    # Get the directory path of the original image
    directory = os.path.dirname(original_image_path)

    # Create the subfolders if they don't exist
    for folder in output_folders.values():
        folder_path = os.path.join(directory, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    # Save the images in the corresponding output folders with the original filename
    red_output_path = os.path.join(directory, output_folders["red"], filename)
    blue_output_path = os.path.join(directory, output_folders["blue"], filename)
    green_output_path = os.path.join(directory, output_folders["green"], filename)

    # Make unspecified pixels black in each channel and save the results in the respective folders
    make_other_pixels_black(original_image_path, red_pixel_locations, red_output_path)
    make_other_pixels_black(original_image_path, blue_pixel_locations, blue_output_path)

    # Since direct ratio test doesn't work, we try keeping all locations that are left out filtering out blue and red
    make_other_pixels_white(original_image_path, np.concatenate([red_pixel_locations, blue_pixel_locations], axis=0), green_output_path)


In [7]:
"""
Generate 3D plot and csv file using processed images above

"""

data = []

# Read in images from corresponding directory
directory = "/mnt/c/Users/alada/JC_Electronics/JC-SDKIntegration/Blue32/thresholded/blue/"

image_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith((".jpg", ".jpeg", ".bmp"))]

for image_file_path in image_files:
    #get the basename of the image file
    filename = os.path.basename(image_file_path)
    
    # Split the filename using underscores
    elements = filename.split('_')

    # Extract the first three elements as RGB values
    red, green, blue = int(elements[0]), int(elements[1]), int(elements[2])
    
    # Attain the contour average intensity
    contour_average_intensity = get_contour_average_intensity(image_file_path, 5)
        
    # If one wishes to fix on a specific color, just don't include that color in the list
    data.append([red, green, contour_average_intensity])

# Extract x, y, and z from data
x = [point[0] for point in data]
y = [point[1] for point in data]
z = [point[2] for point in data]

# Create a 3D figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D scatter plot
ax.scatter(x, y, z)

# Add labels and title
ax.set_xlabel('red')
ax.set_ylabel('green')
ax.set_zlabel('contour average intensity')
ax.set_title('Red vs Green vs Contour Average Intensity')

# Save the plot as an image
plt.savefig(directory+'3d_scatter_plot.png')

# Close the plot to release resources
plt.close()

print("Plot saved as 3d_scatter_plot.png")

# Save data to a CSV file
with open(directory+'Blue32.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Red', 'Green', 'Contour Average Intensity'])  # Write header
    for i in range(len(data)):
        writer.writerow(data[i])
        

Plot saved as 3d_scatter_plot.png


In [ ]:
"""
Generates a 2D scatter plot of the intensities of a white dot in the middle of the image. Plots are saved for analysis.

"""

# directory should include all images one wishes to process
directory = directory + "thresholded/"
    
x = 0
y = 0
image_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith((".jpg", ".jpeg", ".png", ".bmp"))]
for image_file in image_files:
    print(image_file)
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

    # Construct scatter plot of pixel intensities
    # Get the dimensions of the image
    height, width = image.shape[:2]

    if x == 0 or y == 0:
        # Find the middle coordinates
        middle_x = height // 2
        middle_y = width // 2
        contour, centroid = find_contour_closest_to_pixel(image_file, (middle_y,middle_x))
        
        x,y = centroid

    print("Centroid: {}".format(centroid))

    plot_intensity_vs_x(image_file, x, y)
    plot_intensity_vs_y(image_file, x, y)


In [ ]:
"""
Experiment to threshold an image with different values. Csv file is saved for analysis.

"""

# Read image in grayscale
image_path = "/mnt/c/Users/alada/JC_Code/JC-SDKIntegration/Red32/32_32_32_clear_4000.bmp"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = extract_middle_roi(image, 1500)

# Initialize a list to store the results
results = []

for threshold in range(5, 50):
    _, filtered_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    new_path = "/mnt/c/Users/alada/JC_Code/JC-SDKIntegration/ThresholdExperiment/threshold=" + str(threshold) + ".bmp"
#     cv2.imwrite(new_path, filtered_image)
    
    # for each image, find the average, sd, and average + sd, and export them to a csv file
    
    # Perform calculations for black pixels
    black_mask = np.where(image < threshold, 1, 0)
    black_masked_intensities = image[black_mask == 1]
    black_mean_intensity = np.mean(black_masked_intensities)
    black_sd_intensity = np.std(black_masked_intensities)
    black_one_sd = black_mean_intensity + (1 * black_sd_intensity)
    black_two_sd = black_mean_intensity + (2 * black_sd_intensity)
    
    # Perform calculations for white pixels
    white_mask = np.where(image >= threshold, 1, 0)
    white_masked_intensities = image[white_mask == 1]
    white_mean_intensity = np.mean(white_masked_intensities)
    white_sd_intensity = np.std(white_masked_intensities)
    white_one_sd = white_mean_intensity + (1 * white_sd_intensity)
    white_two_sd = white_mean_intensity + (2 * white_sd_intensity)
    
    # Store the results in a dictionary
    result = {
        'Threshold': threshold,
        'Mean Intensity of Black Pixels': black_mean_intensity,
        'Standard Deviation of Black Pixels': black_sd_intensity,
        'Black One SD Above Average': black_one_sd,
        'Black Two SD Above Average': black_two_sd,
        'Mean Intensity of White Pixels': white_mean_intensity,
        'Standard Deviation of White Pixels': white_sd_intensity,
        'White One SD Above Average': white_one_sd,
        'White Two SD Above Average': white_two_sd
    }
    results.append(result)
    
# Define the CSV file path
csv_file_path = '/mnt/c/Users/alada/JC_Code/JC-SDKIntegration/ThresholdExperiment/results.csv'

# Define the fieldnames for the CSV file
fieldnames = [
    'Threshold',
    'Mean Intensity of Black Pixels',
    'Standard Deviation of Black Pixels',
    'Black One SD Above Average',
    'Black Two SD Above Average',
    'Mean Intensity of White Pixels',
    'Standard Deviation of White Pixels',
    'White One SD Above Average',
    'White Two SD Above Average'
]

# Write the results to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for result in results:
        writer.writerow(result)


# Print a confirmation message
print("Results exported to results.csv")